In [1]:
import os
import pandas as pd
import sqlite3 as sql
import gmaps

In [2]:
# make sure to export your API key to shell start up (e.g. ~/.bash_profile)
gmaps.configure(api_key=os.environ["GOOGLE_API_KEY"])

# 1. Load in data

In [3]:
data = pd.read_csv('Data/Police_Department_Incident_Reports__2018_to_Present.csv')
conn = sql.connect("policeDB.db")
#data.to_sql('police', conn)

In [4]:
data.columns

Index(['Incident Datetime', 'Incident Date', 'Incident Time', 'Incident Year',
       'Incident Day of Week', 'Report Datetime', 'Row ID', 'Incident ID',
       'Incident Number', 'CAD Number', 'Report Type Code',
       'Report Type Description', 'Filed Online', 'Incident Code',
       'Incident Category', 'Incident Subcategory', 'Incident Description',
       'Resolution', 'Intersection', 'CNN', 'Police District',
       'Analysis Neighborhood', 'Supervisor District', 'Latitude', 'Longitude',
       'Point', 'Neighborhoods', 'ESNCAG - Boundary File',
       'Central Market/Tenderloin Boundary Polygon - Updated',
       'Civic Center Harm Reduction Project Boundary',
       'HSOC Zones as of 2018-06-05', 'Invest In Neighborhoods (IIN) Areas',
       'Current Supervisor Districts', 'Current Police Districts'],
      dtype='object')

# 2. Some SQL to process data

In [53]:
# 1. using cursor method
cursor = conn.cursor()
x = ('Larceny - From Vehicle',)  # encapsulate to protect from sql injection attacks
sql_query = '''SELECT Latitude, Longitude FROM police WHERE police.'Incident Subcategory'=?'''
cursor.execute(sql_query, x)

In [54]:
print(len(cursor.fetchall()))
cursor.close()

94627


In [5]:
# 2. or using Pandas
sql_query = '''SELECT Latitude, Longitude FROM police WHERE police.'Incident Subcategory'='Larceny - From Vehicle' '''
locations_theft = pd.read_sql(sql_query, conn)

In [6]:
# remove NaN values
locations_theft.dropna(inplace=True)

In [7]:
print(locations_theft.shape)
print(locations_theft.head(5))

(84731, 2)
    Latitude   Longitude
0  37.723944 -122.422830
1  37.782488 -122.445821
2  37.787087 -122.421560
3  37.780615 -122.416138
4  37.730037 -122.404593


# 3. Build Visualization with gmaps

gmaps is built around idea of adding layers to base map

In [8]:
# jupyter nbextension enable --py gmaps
fig = gmaps.figure(layout={
        'width': '600px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black',
        'map_type': 'HYBRID'}, 
        map_type='SATELLITE', center=(37.75,-122.45), zoom_level=12)  # base map

In [9]:
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='600px'))

In [10]:
fig = gmaps.figure(layout={
        'width': '600px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black',
        'map_type': 'HYBRID'}, 
        map_type='SATELLITE', center=(37.75,-122.45), zoom_level=12)  # base map

heatmap_layer = gmaps.heatmap_layer(locations_theft)

heatmap_layer.max_intensity = 80
heatmap_layer.point_radius = 8

fig.add_layer(heatmap_layer)

In [11]:
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='600px'))

In [ ]:
# next up, some marker locations of the high intensity areas.